In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle
from music21 import *
from glob import glob
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.utils import np_utils

%run lstm_model.ipynb
np.set_printoptions(threshold=1000000)
bach_chorale_path = 'datasets/bach'
parts_index = [0,1,2,3]
note_type = set()
"""
note_type:
0-127 : note pitch
128: slur-1st-part
129: slur-2nd-part
130: slur-3rd-part
131: slur-4th-part
132: rest
133: start symbol
134: end symbol
"""

def filter_files(files):
    file_list = []
    file_count = 0
    for i, file in enumerate(files, 1):
        s = converter.parse(file)
        if len(s.parts) == 4:
            print(i, file)
            file_count += 1
            file_list.append(file)
    return file_list, file_count

def chorale_to_inputs(s,i):
    inputs = []
    for n in s.parts[i].flat.notes:
        #print(n.nameWithOctave, int(4 * n.quarterLength), n.offset)
        #print(n.expressions)
        if n.isNote:
            inputs.append(n.pitch.midi)
            note_type.add(n.pitch.midi)
        elif n.isChord:
            inputs.append(n[-1].pitch.midi)
            note_type.add(n[-1].pitch.midi)
        for j in range(1, int(4 * n.quarterLength)):
            if i == 0:
                inputs.append(128)
                note_type.add(128)
            elif i == 1:
                inputs.append(129)
                note_type.add(129)
            elif i == 2:
                inputs.append(130)
                note_type.add(130)
            elif i == 3:
                inputs.append(131)
                note_type.add(131)
    return inputs

def make_time_metadata(file):
    chorale = converter.parse(file)
    chorale_length = int(chorale.parts[0].duration.quarterLength * 4)
    time_metadata = np.zeros((chorale_length, ))
    
    for i in range(0, chorale_length):
        time_metadata[i] = i + 1
    return np.array(time_metadata, dtype=np.int32)

def append_notes(max_length, part_i_length, part_i_input):
    note_type.add(132)# 132-->note.Rest
    for num_appended_notes in range(max_length - part_i_length):
        part_i_input.append(132)
    return part_i_input

def check_parts_length(parts_length, inputs):
    max_length = max(parts_length)
    for i in parts_index:
        if parts_length[i] < max_length:
            inputs[i] = append_notes(max_length, parts_length[i], inputs[i])
                
def make_dataset(file_list):
    X = []
    time_metadata = []
    for file in tqdm(file_list):
        s = converter.parse(file)
        
        inputs = []
        parts_length = []
        time_metadata.append(make_time_metadata(file))
        for i in parts_index:
            part_input = chorale_to_inputs(s,i)
            parts_length.append(len(part_input))
            inputs.append(part_input)
        check_parts_length(parts_length, inputs)
        np_inputs = np.array(inputs)
        X.append(np_inputs)
    #X.reshape(X.shape[1:])
    #print(time_metadata)
    return X
def generator_X(notes):
    t = 0
    while True:
        X = []
        if t < notes.shape[0] - 16:
            X.append(notes[t:t+16])
        else: 
            t = 0
            continue
        t += 16
        X = np.array(X)
        yield (X)
def generate_notes(lstm_model, X):
    output_indices = []
    for i in range(4):
        model = load_model("models/bach"+str(i)+".h5")
#     model = load_model(lstm_model.name+".h5")
        gen = ((X) for (X) in generator_X(lstm_model.X))
#     predict_list = model.predict(X, batch_size=32)
        predict_list = model.predict_generator(gen, steps=500)
        part_notes = []
        for time in range(predict_list.shape[0]):
            for note_index in range(predict_list.shape[1]):
                part_notes.append(np.argmax(predict_list[time][note_index]))
        output_indices.append(part_notes)
    output_indices = np.array(output_indices)
    output_indices = np.transpose(output_indices)
    output_chorale = index_note_transform(output_indices, "to_note", 4)
#     print(output_chorale)
    output_chorale = np.transpose(output_chorale)
    midi = chorale_to_midi(output_chorale)
    print(output_chorale)
    return midi
def chorale_to_midi(chorale):
    score = stream.Score()
    for i, chorale_part in enumerate(chorale):
        part = stream.Part(id=i)
        d = 0
        new_note = note.Rest()
        for n in chorale_part:
            if n == 132:
                part.append(note.Rest())
            elif n < 128:
                if d > 0:
                    new_note.duration = duration.Duration(d / 4)
                    part.append(new_note)
                d = 1
                new_note = note.Note(n)
            else:
                d += 1
        new_note.duration = duration.Duration(d / 4)
        part.append(new_note)
        score.insert(part)
    return score
def make_note_type_list():
    note_type_list = []
    for i in note_type:
        note_type_list.append(i)
    return sorted(note_type_list)
# def transform_note_to_index(notes):
#     for time in range(notes.shape[0]):
#         for p_index in range(4):
#             notes[time][p_index] = note_dict[notes[time][p_index]]
#     return notes
def index_note_transform(inputs, option, pn):
    if option == "to_note":
        note_dict = dict((index, note) for index, note in enumerate(note_type))
    elif option == "to_index":
        note_dict = dict((note, index) for index, note in enumerate(note_type))
        
    for time in range(inputs.shape[0]):
        for p_index in range(pn):
            inputs[time][p_index] = note_dict[inputs[time][p_index]]
    return inputs
def notes_to_piano_roll(dataset_notes):
    start_symbol = np.full((16,4),133)
    end_symbol = np.full((16,4),134)
    note_type.add(133)
    note_type.add(134)
    for chorale_index in range(len(dataset_notes)):
        chorale = np.array(dataset_notes[chorale_index])
        transposed_chorale = np.concatenate((start_symbol,np.transpose(chorale),end_symbol))
        piano_roll_notes = index_note_transform(transposed_chorale, "to_index", 4)
        yield piano_roll_notes
def notes_to_onehot(notes):
    note_dict = note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = []
        for note in n:
            a.append(np.array(note == np.arange(0, len(note_dict)), dtype=np.int32))
        onehot.append(a)
    return np.array(onehot)
def part_notes_to_onehot(notes):
    note_dict = note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = np.zeros(len(note_dict,),dtype=np.int32)
        for note in n:
            a[note] = 1
        onehot.append(a)
    return np.array(onehot)

def load_files():
    if os.path.exists(bach_chorale_path):
#         file_list = filter_files(glob(bach_chorale_path))
        file_list, num_file = filter_files(glob(bach_chorale_path + '/*.mxl') +
                                 glob(bach_chorale_path + '/*.mid'))
        print(num_file)
        dataset_notes = make_dataset(file_list)
        shuffle(dataset_notes)
        chorale_list = [notes for chorale_index, notes in enumerate(notes_to_piano_roll(dataset_notes))]
        notes = chorale_list[0]
        for i, n in enumerate(chorale_list):
            if i != 0:
                notes = np.concatenate((notes, n))
        Y_onehot_notes = notes_to_onehot(notes)
        X_onehot_notes = part_notes_to_onehot(notes)
#         notes = notes[:,:,np.newaxis]
        for i in range(4):
            lstm_model = LSTM_Model(X=X_onehot_notes,Y=Y_onehot_notes,epochs=5, output_dim=len(note_type), name="bach"+str(i))
            gen = ((X,Y) for (X, Y) in lstm_model.generator(X_onehot_notes, Y_onehot_notes, i))
            lstm_model.model2()
            lstm_model.train_model(gen)

        score = generate_notes(lstm_model, notes)
                
        output_file = str(lstm_model.epochs) + '-epochs_' + str(num_file) + '-samples.mid'
        mf = midi.translate.music21ObjectToMidiFile(score)
        mf.open(output_file, 'wb')
        mf.write()
        mf.close()
        print("File " + output_file + " written")
#         build_autoEncoder(onehot_notes)
#         model = train_data(notes)
#         predict_notes(notes, model)
    else:
        print("The file path is wrong!!")
    
def main():
    load_files()
if __name__ == "__main__":
    main()

Using TensorFlow backend.


1 datasets/bach2\bwv339.mxl
1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.61it/s]


Another model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 36)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 16, 200)           189600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 200)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 16, 200)           40200     
_________________________________________________________________
dense_2 (Dense)              (None, 16, 36)            7236      
Total params: 237,036
Trainable params: 237,036
Non-trainable params: 0
_________________________________________________________________
train model...
Epoch 1/5
 36/256 [===>..........................] - ETA: 238s - loss: 3.596 - ETA: 83s - loss: 3.545 - 

KeyboardInterrupt: 

In [ ]:
n = [0,1,2,3,4,5,6,7,8]
n = np.array(n)
g = np.zeros((len(n),),dtype=np.int32)
print(g)
a = [0,5,7]
for i in a:
    print(np.array(i == np.arange(0, len(n) )))

In [86]:
def inner_pro(x1,x2,y1,y2):
    return (x1*y1)+(x2*y2)
x = [1.4, 1.6]
A1 = [1.5, 2.0, 1.6, 1.2, 1.5]
A2 = [1.7, 1.9, 1.8, 1.5, 1.0]
ans = []
for i in range(5):
    ans.append(inner_pro(x[0],x[1],A1[i],A2[i]) / (inner_pro(x[0],x[1],x[0],x[1]) * inner_pro(A1[i],A2[i],A1[i],A2[i])))
ans

[0.2074653076684687,
 0.16978126126545184,
 0.19530057979859625,
 0.2446219152457011,
 0.2518720217835262]

0.5